<a href="https://colab.research.google.com/github/lucascheng24/COMP4431_LabAssignment/blob/main/A7/assignment_7_keras.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

import os
import cv2

import pandas as pd
from PIL import Image
from typing import Any, Tuple, Optional, Callable
from torch.utils.data import Dataset

def read_csv(path: str) -> pd.DataFrame:
    '''
    Read a csv file.

    Args:
        path (str): Path to the csv file.

    Returns:
        pd.DataFrame: Dataframe with the csv file data.
    '''

    assert os.path.exists(path), f'CSV file not found: {path}!'
    assert os.path.splitext(path)[
    -1] == '.csv', f'Unsupported file type {os.path.splitext(path)[-1]}!'
    return pd.read_csv(path)

class ImageDataset(Dataset):
    def __init__(self, dataframe: pd.DataFrame, images_folder: str = './images', transform: Optional[Callable] = None, target_transform: Optional[Callable] = None) -> None:
        '''
        Image dataset.

        Args:
            dataframe (pd.DataFrame): Dataframe with the image filenames and labels.
            images_folder (str): Directory with all the images.
            transform (callable, optional): Optional transform to be applied on a sample.
            target_transform (callable, optional): Optional transform to be applied on a target.
        '''
        assert 'Filename' in dataframe.columns, f'Filename column not found!'
        assert os.path.exists(images_folder), f'Image folder not found: {images_folder}!'

        self.dataframe = dataframe
        self.images_folder = images_folder
        self.transform = transform
        self.target_transform = target_transform

        data = []
        targets = []

        for i, sample in dataframe.iterrows():
            image = cv2.imread(os.path.join(images_folder, sample['Filename']))
            data.append(image)

            targets.append(int(sample['Label']) if 'Label' in sample else -1)

        self.data = data
        self.targets = targets

    def __len__(self) -> int:
        '''
        Returns:
            int: Length of the dataset.
        '''
        return len(self.data)

    def __getitem__(self, index: int) -> Tuple[Any, Any]:
        '''
        Args:
            index (int): Index

        Returns:
            tuple: (image, target) where target is class_index of the target class. For the public test set, target is a class from [0, 1, 2, 3, 4, 5, 6, 7, 8]. For the private test set (before releasing the test set labels), target is -1.
        '''
        img = self.data[index]
        target = self.targets[index]

        img = Image.fromarray(img)

        if self.transform is not None:
            img = self.transform(img)

        if self.target_transform is not None:
            target = self.target_transform(target)

        return img, target

In [ ]:
public_dataframe = read_csv('assignment_7_public.csv')
public_dataset = ImageDataset(public_dataframe)

print('Image', type(public_dataset[0][0]), public_dataset[0][0].size) # Image <class 'PIL.Image.Image'> (28, 28)
print('Target', type(public_dataset[0][1])) # Target <class 'int'>
print('Length', len(public_dataset)) # Length 85744

'''
CODE HERE!
'''


Image <class 'PIL.Image.Image'> (28, 28)
Target <class 'int'>
Length 85744


'\nCODE HERE!\n'

In [ ]:
from tensorflow import keras
from sklearn.model_selection import train_test_split
import numpy as np

# Assuming the label column is named 'label' in public_dataframe
labels = np.array(public_dataframe['Label'])

# Extract the image data from the ImageDataset
images = np.asarray([np.array(sample[0]) for sample in public_dataset])

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=42)

# Normalize the input data (assuming it is in the range of 0-255)
X_train = X_train.astype('float32') / 255.0
X_test = X_test.astype('float32') / 255.0

input_shape = (28, 28, 3)

# Define the model architecture
model = keras.Sequential(
    [
        keras.Input(shape=input_shape),
        keras.layers.Conv2D(32, kernel_size=(3, 3), activation="relu"),
        keras.layers.MaxPooling2D(pool_size=(2, 2)),
        keras.layers.Conv2D(64, kernel_size=(3, 3), activation="relu"),
        keras.layers.MaxPooling2D(pool_size=(2, 2)),
        keras.layers.Conv2D(128, kernel_size=(3, 3), activation="relu"),
        keras.layers.MaxPooling2D(pool_size=(2, 2)),
        keras.layers.Flatten(),
        keras.layers.Dropout(0.5),
        keras.layers.Dense(9, activation="softmax"),
    ]
)

model.compile(optimizer='Adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

batch_size = 128
epochs = 30
# Train the model
model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, validation_data=(X_test, y_test))

# Evaluate the model on the test data
_, accuracy = model.evaluate(X_test, y_test)

# Print the accuracy
print("Accuracy:", accuracy)


Epoch 1/30
536/536 [==============================] - 3s 6ms/step - loss: 1.5319 - accuracy: 0.4261 - val_loss: 1.1533 - val_accuracy: 0.5709
Epoch 2/30
536/536 [==============================] - 3s 5ms/step - loss: 1.1437 - accuracy: 0.5798 - val_loss: 0.9393 - val_accuracy: 0.6635
Epoch 3/30
536/536 [==============================] - 3s 5ms/step - loss: 1.0113 - accuracy: 0.6316 - val_loss: 0.8467 - val_accuracy: 0.6997
Epoch 4/30
536/536 [==============================] - 3s 5ms/step - loss: 0.9303 - accuracy: 0.6610 - val_loss: 0.8401 - val_accuracy: 0.6971
Epoch 5/30
536/536 [==============================] - 3s 5ms/step - loss: 0.8851 - accuracy: 0.6786 - val_loss: 0.7510 - val_accuracy: 0.7283
Epoch 6/30
536/536 [==============================] - 3s 5ms/step - loss: 0.8586 - accuracy: 0.6873 - val_loss: 0.7660 - val_accuracy: 0.7186
Epoch 7/30
536/536 [==============================] - 3s 5ms/step - loss: 0.8182 - accuracy: 0.7037 - val_loss: 0.7262 - val_accuracy: 0.7379
Epoch 

In [ ]:
private_dataframe = read_csv('assignment_7_private.csv')
private_dataset = ImageDataset(private_dataframe)

print('Image', type(private_dataset[0][0])) # Image <class 'PIL.Image.Image'> (28, 28)
print('Length', len(private_dataset)) # Length 21436

import numpy as np

# remove and make your own predictions.
preds = np.full(len(private_dataset), -1,
                dtype=int)
'''
CODE HERE!
e.g.,
preds = np.full(len(X_private), -1, dtype=int)
'''

# Convert images to numpy arrays and normalize
predict_imgages = np.asarray([np.array(sample[0]) for sample in private_dataset])
predict_imgages = predict_imgages.astype('float32') / 255.0

# Make predictions using the trained model
predicted_labels = model.predict(predict_imgages)

# Get the predicted labels
preds = np.argmax(predicted_labels, axis=1)


submission = pd.DataFrame({'Label': preds})
submission.to_csv('assignment_7.csv', index=True, index_label='Id')

Image <class 'PIL.Image.Image'>
Length 21436
670/670 [==============================] - 1s 1ms/step
